In [232]:
# Imports

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import random

from sklearn.utils import check_random_state


In [233]:
"""# Set random seed for NumPy
np.random.seed(42)
# Set random seed for Python's built-in random module
random.seed(42)
# Set global seed for scikit-learn's check_random_state (if applicable)
check_random_state(42)"""

"# Set random seed for NumPy\nnp.random.seed(42)\n# Set random seed for Python's built-in random module\nrandom.seed(42)\n# Set global seed for scikit-learn's check_random_state (if applicable)\ncheck_random_state(42)"

# Data analisys

In [234]:
activities_df=pd.read_csv("../data/activitats.csv",  encoding="ISO-8859-1")
marks_df=pd.read_csv("../data/notes.csv", sep=";")
submissions_df=pd.read_csv("../data/trameses.csv")


In [235]:
activities_df.head()

,activitat_id,activitat,aula_id,startdate,duedate,grade
0,3,Problema 1.1 - Hello world!!!,87,0,0,100
1,4,Problema 1.2 - Hello world!!! ++,87,0,0,100
2,5,Problema 3a.1: Càlcul edat,87,0,0,100
3,6,Problema 3a.8: Conversió dòlars a euros,87,0,0,100
4,7,Problema 3a.11: Mitjana de notes,87,0,0,100


In [236]:
activities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1801 entries, 0 to 1800
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   activitat_id  1801 non-null   int64 
 1   activitat     1801 non-null   object
 2   aula_id       1801 non-null   int64 
 3   startdate     1801 non-null   int64 
 4   duedate       1801 non-null   int64 
 5   grade         1801 non-null   int64 
dtypes: int64(5), object(1)
memory usage: 84.6+ KB


In [237]:
marks_df.head()

,userid,aula_id,P_Grade,P_Grade_Date,F_Grade,F_Grade_Date,R_Grade,R_Grade_Date
0,176,92,NaN,NaN,"9,25",1667829600,NaN,1675173600
1,179,92,NaN,NaN,"7,75",1667829600,NaN,1675173600
2,168,92,NaN,NaN,"7,75",1667829600,NaN,1675173600
3,401,92,NaN,NaN,"3,50",1667829600,"8,25",1675173600
4,482,92,NaN,NaN,"7,00",1667829600,NaN,1675173600


In [238]:
submissions_df.head()

,id,activitat_id,userid,datesubmitted,grader,dategraded,grade,nevaluations
0,6,3,31,1658829978,0.0,1.658830e+09,100.0,1
1,7,362,26,1658837049,0.0,1.658837e+09,100.0,0
2,9,362,26,1658837075,0.0,1.658837e+09,100.0,0
3,15,362,31,1662537577,0.0,1.662538e+09,NaN,0
4,16,362,31,1662537593,0.0,1.662712e+09,100.0,0


In [239]:
from pandasql import sqldf
print(sqldf("SELECT * from submissions_df WHERE id=7"))

   id  activitat_id  userid  datesubmitted  grader    dategraded  grade  \
0   7           362      26     1658837049     0.0  1.658837e+09  100.0   

   nevaluations  
0             0  


## Data cleaning

### Delete unnecesary variables

In [240]:
# Dates are unnecessary information for our goal
activities_df = activities_df.drop('startdate', axis=1)
activities_df = activities_df.drop('duedate', axis=1)

marks_df = marks_df.drop('P_Grade_Date', axis=1)
marks_df = marks_df.drop('F_Grade_Date', axis=1)
marks_df = marks_df.drop('R_Grade_Date', axis=1)

submissions_df = submissions_df.drop('datesubmitted', axis=1)
submissions_df = submissions_df.drop('dategraded', axis=1)

#submissions_df = submissions_df.drop('grader', axis=1) #Also a useles column


### Delete missing data

In [241]:
print(activities_df.isnull().sum())
print()
print(marks_df.isnull().sum())
print()
print(submissions_df.isnull().sum())


activitat_id    0
activitat       0
aula_id         0
grade           0
dtype: int64

userid       0
aula_id      0
P_Grade    867
F_Grade    381
R_Grade    838
dtype: int64

id                   0
activitat_id         0
userid               0
grader               7
grade           253347
nevaluations         0
dtype: int64


In [242]:
activities_df['aula_id'].value_counts()

141    262
184    232
126    215
87     213
176    176
143    175
114    146
125    146
92     145
164     75
165     15
170      1
Name: aula_id, dtype: int64

In [243]:
marks_df['aula_id'].value_counts()

92     252
176    248
143    238
184    125
141    113
87     112
Name: aula_id, dtype: int64

In [244]:
submissions_df['userid'].value_counts()

408     2761
2149    2556
158     2485
1653    2369
1670    1970
        ... 
1643       4
2244       4
2602       4
2492       3
240        2
Name: userid, Length: 1021, dtype: int64

In [245]:
# We delete the column recuperation
marks_df = marks_df.drop('R_Grade', axis=1)

### Filling missing values

In [246]:
submissions_df['grader'].fillna(submissions_df['grader'].median(), inplace=True)

submissions_df['grade'].fillna(0, inplace=True)

### Add new variables

In [247]:
# We create a new boolean variable to ask wheter the student has made the partial exam or not
marks_df['made_partial'] = marks_df['P_Grade'].isnull()
marks_df = marks_df.drop('P_Grade', axis=1) # We drop this column as it is not useful anymore

To finish with the cleaning part, we will create a new csv, the one we will use to train the model. Here we will create new statistic variables about the exercices, to make the data abaliable for the model.

In [248]:
users_df=sqldf("select distinct s.userid, s.grader, s.grade, s.nevaluations, m.aula_id, a.*, m.F_Grade from activities_df as a inner join submissions_df as s on a.activitat_id=s.activitat_id inner join marks_df as m on s.userid=m.userid")

users_df.head()

,userid,grader,grade,nevaluations,aula_id,activitat_id,activitat,aula_id,grade,F_Grade
0,227,0.0,100.0,1,87,3,Problema 1.1 - Hello world!!!,87,100,"9,5"
1,229,0.0,100.0,1,87,3,Problema 1.1 - Hello world!!!,87,100,"10,5"
2,241,0.0,100.0,1,87,3,Problema 1.1 - Hello world!!!,87,100,"9,25"
3,279,0.0,0.0,0,87,3,Problema 1.1 - Hello world!!!,87,100,"7,25"
4,279,0.0,100.0,1,87,3,Problema 1.1 - Hello world!!!,87,100,"7,25"


In [249]:
q1="select * from user_df where nevaluations=(select max(nevaluations) from user_df group by userid) group by userid, activitat_id"

# Get only the rows for user and activity where the nevaluations is max
q2="SELECT t.* FROM user_df t JOIN (SELECT userid, activitat_id, MAX(nevaluations) AS max_nevaluations FROM user_df GROUP BY userid, activitat_id) AS max_vals ON t.userid = max_vals.userid AND t.activitat_id = max_vals.activitat_id AND t.nevaluations = max_vals.max_nevaluations"
users_df=sqldf(q2)
users_df = users_df.dropna(subset=['F_Grade'])
users_df.head()

,userid,grader,grade,nevaluations,aula_id,activitat_id,activitat,F_Grade
0,227,0.0,100,1,87,3,Problema 1.1 - Hello world!!!,"9,5"
1,229,0.0,100,1,87,3,Problema 1.1 - Hello world!!!,"10,5"
2,241,0.0,100,1,87,3,Problema 1.1 - Hello world!!!,"9,25"
3,279,0.0,100,1,87,3,Problema 1.1 - Hello world!!!,"7,25"
4,292,0.0,100,1,87,3,Problema 1.1 - Hello world!!!,7


In [250]:
single_user_df=sqldf("select distinct * from users_df where userid=408")
single_user_df.head()

,userid,grader,grade,nevaluations,aula_id,activitat_id,activitat,F_Grade
0,408,0.0,100,1,92,362,Problema 1.1 - Hello world!!!,"8,00"
1,408,0.0,100,3,92,363,Problema 1.2 - Hello world!!! ++,"8,00"
2,408,0.0,100,4,92,366,Problema 3a.1: Càlcul edat,"8,00"
3,408,0.0,100,4,92,367,Problema 3a.8: Conversió dòlars a euros,"8,00"
4,408,0.0,100,6,92,368,Problema 3a.11: Mitjana de notes,"8,00"


In [251]:
activities_df.to_csv("../data/activities_clean.csv", index=False)
marks_df.to_csv("../data/marks_clean.csv", index=False)
submissions_df.to_csv("../data/submissions_clean.csv", index=False)

single_user_df.to_csv("../data/single_user.csv", index=False)
users_df.to_csv("../data/users_clean_vApplication.csv", index=False)

## Create statistic variables about activities

In [252]:
q1="select distinct userid, grader, aula_id, min(grade) as min_grade, max(grade) as max_grade, avg(grade) as avg_grade, min(nevaluations) as min_nevaluations, max(nevaluations) as max_nevaluations, avg(nevaluations) as avg_nevaluations, count(distinct activitat_id) as num_activities, F_Grade from users_df group by userid order by userid"
q2="select distinct userid, count(activitat_id) from user_df group by userid"
users_df=sqldf(q1)
print(users_df)
print(sqldf(q2))

     userid  grader  aula_id  min_grade  max_grade   avg_grade  \
0        21     0.0       92        100        100  100.000000   
1        35     0.0       92        100        100  100.000000   
2        57     0.0      143          1        100   54.222857   
3        65     0.0       92        100        100  100.000000   
4        70     0.0       92        100        100  100.000000   
..      ...     ...      ...        ...        ...         ...   
663    2253     0.0      143          1        100   28.722892   
664    2271     0.0      143          1        100   20.568966   
665    2272     0.0      141        100        100  100.000000   
666    2273     0.0      141        100        100  100.000000   
667    2415     0.0      176        100        100  100.000000   

     min_nevaluations  max_nevaluations  avg_nevaluations  num_activities  \
0                   0                17          3.123596              82   
1                   1                43          3.52

In [253]:
users_df.to_csv("../data/users_clean_vModel.csv", index=False)

## Data preprocessing

In [254]:
#? User info: class, marks, list of activities (dependent on the class) 
# Para que todos los usuarios tengan la misma length puedo tomar otras variables estadisticas, como por ejemplo, el numero de ejercicios completados, el numero medio, mayor y menor de intentos por ejercicio, etc...
# Users clean lo hare por dos
